# AIS regularize script documentation

## Introduction

Before going to the regularization of the Automatic Identification System (AIS) data this document starts with the introduction of the format of the data. Some recommendations on what additional data can be obtained from the "raw" AIS data that are not available in our data set are provided. Also some background information on the data is provided. If the reader is familiar with the data they can skip this section and go on reading the script commentary that starts with the subsection _"Reading the AIS data"_. The AIS data was collected for the purposes of the BIAS project (https://biasproject.wordpress.com/). The recommendations and comments are most useful for people wanting to use the AIS data for recorded underwater sound analysis.

## Format of the BIAS AIS data

The AIS data for all the sound monitoring locations for the year 2014 was collected and saved in the _.txt_ file format. For every monitoring location exists 12 of the files, one for each month of the year. The first few lines of the _.txt_ file look as following

DAY;TIME;MMSI;LAT;LON;Vs;TYPEc;L1;L2;B1;B2;TYPEs;DRAUGHT;BIASST;DISTKM
01-01-2014;18:02:17;310640000;55.205825;13.31618;15;70;190;39;19;13;17;7;2;19.1188469545512
01-01-2014;18:01:04;310640000;55.20629167;13.30748833;15;70;190;39;19;13;17;7;2;18.6783675948067
01-01-2014;17:57:59;310640000;55.20743;13.28499167;15;70;190;39;19;13;17;7;2;17.5740792317342
01-01-2014;17:53:28;310640000;55.20901167;13.252135;15;70;190;39;19;13;17;7;2;16.0628887633561
...

Which in a table format looks like

|        DAY |     TIME |      MMSI |         LAT |         LON | Vs | TYPEc |  L1 | L2 | B1 | B2 | TYPEs | DRAUGHT | BIASST |           DISTKM |
|------------|----------|-----------|-------------|-------------|----|-------|-----|----|----|----|-------|---------|--------|------------------|
| 01-01-2014 | 18:02:17 | 310640000 |   55.205825 |    13.31618 | 15 |    70 | 190 | 39 | 19 | 13 |    17 |       7 |      2 | 19.1188469545512 |
| 01-01-2014 | 18:01:04 | 310640000 | 55.20629167 | 13.30748833 | 15 |    70 | 190 | 39 | 19 | 13 |    17 |       7 |      2 | 18.6783675948067 |
| 01-01-2014 | 17:57:59 | 310640000 |    55.20743 | 13.28499167 | 15 |    70 | 190 | 39 | 19 | 13 |    17 |       7 |      2 | 17.5740792317342 |
| 01-01-2014 | 17:53:28 | 310640000 | 55.20901167 |   13.252135 | 15 |    70 | 190 | 39 | 19 | 13 |    17 |       7 |      2 | 16.0628887633561 |

The raw data contains various different data columns, that can be divided into three separate categories:
 - __Instantaneous__ position and speed of a vessel (__DAY, TIME, LAT, LON, Vs, DISTKM__)
 - __Voyage related__ data of the vessel (__DRAUGHT__)
 - __Vessel related__ data (__MMSI, TYPEc, L1, L2, B1, B2__)

The meaning and usefulness of the __TYPEs__ is obscure. The __BIASST__ corresponds to the monitoring location number around which the AIS data is from. Here we will go through the different categories and the data within them.

### Vessel related data

- __MMSI__ is the Maritime Mobile Service Identity of the vessel (https://en.wikipedia.org/wiki/Maritime_Mobile_Service_Identity). This is sort of like a phone number of a vessel (sometimes not only a vessel), as it also entails the code of the country where the ship is registred. The MMSI of a vessel can change. For example when they change their country of registration.
- __TYPEc__ is the two numbered code for the type of the vessel. It can be decoded to give name of the type of the ship. These categories can be somewhat broad.
- __L1, L2, B1, B2__ give the location of the AIS transponde onboard the ship by giving in meters
     - __L1__ Dimension to Bow
     - __L2__ Dimension to Stern
     - __B1__ Dimension to Port
     - __B2__ Dimension to Starboard

From these dimensions the length and width of the ships can be calculated.

If truly "raw" AIS data could be acquired I would recommend also to collect the __IMO number__ of the vessels. When the MMSI corresponds to the phone number the IMO number corresponds to the car's vehicle identification number (VIN). It is something remains linked to the hull for its lifetime, regardless of a change in name, flag, or owner. 

(__Other vessel related data__ available in AIS are the __Callsign__ of the ship and the __Vessel Name__. But also __EPFD no__ or the electronic position fixing device - Glonass or GPS for example.)

### Voyage related data

The only voyage related data availabe in this AIS data set is the __DRAUGHT__. This can be different for every voyage of the vessel. The draught of a ship's hull is the vertical distance in meters between the waterline and the bottom of the hull (keel), with the thickness of the hull included. 

(__Other voyage related data__ available in AIS are the __Destination__ of the ship and the __Estimated Time of arrival__ to the destination, although I have found these lacking in many of the vessels and also not necessary for my purposes.)

### Instantaneous AIS data

- __DAY, TIME__ gives the timestamp of the vessels position in day-month-year Hour:Minute:Second format. The timestamp is in __UTC__ Coordinated Universal Time.
- __LAT, LON__ the latitude and the longitude of the vessel in the World Geodetic System (WGS) coordinates.
- __Vs__ is the speed over ground of the vessel.
- __DISTKM__ is something not provided by AIS but a calculated distance in km to the sound monitoring location. This was probably added by a person dealing with the gathering of the AIS data in the BIAS project. This was not always calculated from the right location and the precision was left unreasonably high (tenth of a nanometer).

I would not recommend on collecting the other instantaneous data as they can be missing and unreliable.

(__Other instantaneous data__ available in AIS are the Rate of Turn __ROT__, Course Over Ground __COG__, True Heading __HDG__, __Maneuver Indicator__, __Position accuracy__)

## Reading the AIS data

Initially the monthly AIS data is just read and added into one pandas dataframe. The __TYPEs__ and the __BIASST__ are not used. The __DAY__ and __TIME__ are added together to give a single timestamp. The names of variables are also changed.

In [3]:
import pandas as pd  # Import Pandas package (high-performance, easy-to-use data structures and analysis tools)
import os  # Provides portable way of using operating system dependent functionality
#import numpy as np  # NumPy is the fundamental package for scientific computing with Python

AIS_dir = 'AIS_BIASstation'  # Directory with all the folders with AIS data from monitoring locations
AIS_mon_locs = os.listdir(AIS_dir)  # List the subdirectories in the AIS_dir folder

# For documentation AIS data from only one monitoring location is sufficient. 
# The script can be simply extended for looping through all the monitoring locations
# by adding "for loc in AIS_mon_locs:" instead of the next line.
loc = AIS_mon_locs[0]  # For demonstration choose only one folder for of AIS data
AIS_mon_loc_path = os.path.join(AIS_dir, loc)  # List monthly AIS data files in the one location AIS folder
# Loop through the monthly AIS data files
for file in os.listdir(AIS_mon_loc_path):
    # Read only the .txt files
    if file.endswith(".txt"):
        file = os.path.join(AIS_mon_loc_path, file)  # Construct the path of the file to be read
        data = pd.read_table(file, sep=';')  # Read the data from the file
        DAYTIME = data.DAY + ' ' + data.TIME  # Add the date and time together into one timestamp
        # Construct a new dataframe from the raed data
        AIS_mon_data = pd.DataFrame(data={'Time': DAYTIME, 'Latitude': data.LAT, 'Longitude': data.LON,
                                          'MMSI': data.MMSI,  'Vs': data.Vs, 'Ship_type': data.TYPEc,
                                          'L1': data.L1, 'L2': data.L2, 'B1': data.B1, 'B2': data.B2,
                                          'Draught': data.DRAUGHT, 'Dist': round(data.DISTKM)})
        # Convert the timestamp to the datetime format
        AIS_mon_data.Time = pd.to_datetime(AIS_mon_data.Time, format='%d-%m-%Y %H:%M:%S', utc=True)
        # Add the AIS data from one location together into one variable
        if file == os.path.join(AIS_mon_loc_path, os.listdir(AIS_mon_loc_path)[0]):
            AIS_mon_data_l = AIS_mon_data  # Create a file to append the data to from the first iteration
        else:
            AIS_mon_data_l = AIS_mon_data_l.append(AIS_mon_data)  # Append to the data frame

del data, AIS_mon_data, DAYTIME, file  # Do some clean up of not used variables

AIS_mon_data_l

,B1,B2,Dist,Draught,L1,L2,Latitude,Longitude,MMSI,Ship_type,Time,Vs
0,6.0,8.0,19.0,7.0,81.0,8.0,55.283667,12.795667,245717000,70.0,2014-12-01 16:39:29+00:00,11
1,6.0,8.0,18.0,7.0,81.0,8.0,55.280000,12.812667,245717000,70.0,2014-12-01 16:36:09+00:00,11
2,6.0,8.0,18.0,7.0,81.0,8.0,55.276500,12.827833,245717000,70.0,2014-12-01 16:33:09+00:00,11
3,6.0,8.0,17.0,7.0,81.0,8.0,55.273167,12.844167,245717000,70.0,2014-12-01 16:29:58+00:00,11
4,6.0,8.0,16.0,7.0,81.0,8.0,55.270000,12.858667,245717000,70.0,2014-12-01 16:27:08+00:00,11
5,6.0,8.0,15.0,7.0,81.0,8.0,55.267000,12.872000,245717000,70.0,2014-12-01 16:24:25+00:00,11
6,6.0,8.0,15.0,7.0,81.0,8.0,55.266833,12.872833,245717000,70.0,2014-12-01 16:24:17+00:00,11
7,6.0,8.0,15.0,7.0,81.0,8.0,55.266833,12.874333,245717000,70.0,2014-12-01 16:24:01+00:00,11
8,6.0,8.0,14.0,7.0,81.0,8.0,55.266167,12.890833,245717000,70.0,2014-12-01 16:20:59+00:00,11
9,6.0,8.0,13.0,7.0,81.0,8.0,55.265333,12.908000,245717000,70.0,2014-12-01 16:17:49+00:00,11


## Mark AIS data from a vessels single voyage

Taking the MMSI code as an identifier of ships the AIS data are divided to be from single ships.

In [ ]:
import datetime  # Supplies classes for manipulating dates and times in both simple and complex ways.
import matplotlib.pyplot as plt  # Provides a MATLAB-like plotting framework.

AIS_mon = AIS_mon_data_l['MMSI'].value_counts() > 1  # Compose a logical list of  whether MMSI only occurs once or more times during a year
ships = AIS_mon_data_l.MMSI.unique()  # Compose a list of unique MMSI numbers for a 

#for one_ship in ships:
#    AIS_mon_one_ship = AIS_mon_data[AIS_mon_data.MMSI == one_ship]

ind = 1
time_thresh = datetime.timedelta(minutes=20)

if AIS_mon[ships[ind]]:
    AIS_mon_one_ship = AIS_mon_data[AIS_mon_data.MMSI == ships[ind]]
    AIS_mon_one_ship = AIS_mon_one_ship.sort_values(by='Time')
    
    plt.plot(AIS_mon_one_ship.Longitude, AIS_mon_one_ship.Latitude, '.')
    plt.show()
    no = 1
    voyage_no = [1]
    for i in range(1, len(AIS_mon_one_ship)):
        
        if AIS_mon_one_ship.Time[i] - AIS_mon_one_ship.Time[i-1] > time_thresh:
            no = no+1
        voyage_no.append(no)
    
    AIS_mon_one_ship['Voyage'] = pd.Series(voyage_no, index=AIS_mon_one_ship.index) 